In [1]:
import numpy as np
import pandas as pd
import calendar
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import classification_report,confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,LSTM,Embedding
from keras import optimizers
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [2]:
# preprocess=pd.read_csv('preprocess.csv', engine='python',index_col="arrival_date")
train=pd.read_csv('preprocess_all/preprocess_train_num_all.csv', engine='python',index_col="arrival_date")
train=train.drop(["quatity"],axis=1)
train

,hotel_Resort,hotel_City,lead_time,arrival_date_week_number,stays_nights,adults,children,babies,meal_BB,meal_FB,...,by_other_booking,wait_less_than_3,wait_between_3_30,wait_greater_than_30,contract,group,transient,transient_party,mean_required_car_parking_space,special_requests_ratio
arrival_date,,,,,,,,,,,,,,,,,,,,,
2015/7/1,38,65,190.291262,27,3.048544,1.805825,0.019417,0.000000,35,1,...,6,103,0,0,6,0,97,0,0.038835,0.184466
2015/7/2,35,1,69.944444,27,5.694444,1.972222,0.055556,0.000000,27,1,...,4,36,0,0,13,0,20,3,0.138889,0.500000
2015/7/3,27,10,65.378378,27,4.324324,2.000000,0.081081,0.000000,22,0,...,1,37,0,0,6,0,22,9,0.189189,0.243243
2015/7/4,36,9,120.777778,27,5.311111,1.977778,0.111111,0.044444,37,1,...,4,45,0,0,7,0,26,12,0.155556,0.488889
2015/7/5,37,0,75.378378,28,6.108108,1.945946,0.216216,0.000000,26,0,...,5,37,0,0,9,1,27,0,0.351351,0.378378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017/3/27,25,89,96.368421,13,2.982456,1.842105,0.000000,0.000000,99,0,...,4,114,0,0,0,2,67,45,0.070175,0.675439
2017/3/28,33,34,52.537313,13,3.820896,1.791045,0.044776,0.014925,54,0,...,5,67,0,0,0,1,64,2,0.059701,0.492537
2017/3/29,51,79,41.130769,13,2.800000,1.653846,0.053846,0.015385,104,0,...,6,130,0,0,0,3,125,2,0.100000,0.484615


In [3]:
scaler = StandardScaler()
scaler.fit(train)
train=scaler.transform(train)
train

array([[ 0.09969601,  0.33131817,  3.07120569, ..., -0.88437064,
        -1.40846684, -2.13684133],
       [-0.08524731, -1.89194401,  0.16456566, ..., -0.77582422,
         0.61368625,  0.06435518],
       [-0.57842948, -1.57929776,  0.05428514, ..., -0.55873138,
         1.63028699, -1.72680556],
       ...,
       [ 0.90111703,  0.81765678, -0.53134789, ..., -0.81200636,
        -0.17228268, -0.04296943],
       [ 0.90111703,  0.60922595, -0.09645553, ..., -0.19690999,
        -0.72644512,  0.12061405],
       [ 1.70253806,  0.71344136, -0.12261841, ..., -0.81200636,
        -0.94961459,  0.70830285]])

In [4]:
y_label=pd.read_csv('train_label.csv', engine='python',index_col="arrival_date")
# test_nolabel = pd.read_csv('test_nolabel.csv', engine='python')
y_label_tr=pd.get_dummies(y_label, columns=["label"])
y_label_tr

,label_0.0,label_1.0,label_2.0,label_3.0,label_4.0,label_5.0,label_6.0,label_7.0,label_8.0,label_9.0
arrival_date,,,,,,,,,,
2015-07-01,0,0,1,0,0,0,0,0,0,0
2015-07-02,0,1,0,0,0,0,0,0,0,0
2015-07-03,0,1,0,0,0,0,0,0,0,0
2015-07-04,0,1,0,0,0,0,0,0,0,0
2015-07-05,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2017-03-27,0,0,1,0,0,0,0,0,0,0
2017-03-28,0,1,0,0,0,0,0,0,0,0
2017-03-29,0,0,1,0,0,0,0,0,0,0


In [6]:
# preprocess=pd.read_csv('preprocess.csv', engine='python',index_col="arrival_date")
test=pd.read_csv('preprocess_all/preprocess_test_num_all.csv', engine='python',index_col="arrival_date")
test=test.drop(["quatity"],axis=1)
test

,hotel_Resort,hotel_City,lead_time,arrival_date_week_number,stays_nights,adults,children,babies,meal_BB,meal_FB,...,by_other_booking,wait_less_than_3,wait_between_3_30,wait_greater_than_30,contract,group,transient,transient_party,mean_required_car_parking_space,special_requests_ratio
arrival_date,,,,,,,,,,,,,,,,,,,,,
2017/4/1,35,97,95.265152,13,3.969697,2.068182,0.151515,0.007576,106,0,...,3,129,0,3,1,0,106,25,0.053030,0.409091
2017/4/2,29,67,73.364583,14,3.822917,1.947917,0.083333,0.010417,73,0,...,2,95,0,1,4,0,88,4,0.052083,0.520833
2017/4/3,64,167,76.995671,14,3.307359,1.761905,0.082251,0.004329,203,0,...,11,229,0,2,4,3,175,49,0.038961,0.415584
2017/4/4,28,97,104.544000,14,3.048000,1.912000,0.152000,0.000000,111,0,...,2,123,2,0,0,0,111,14,0.056000,0.360000
2017/4/5,91,161,119.769841,14,3.484127,1.702381,0.051587,0.000000,232,0,...,21,247,1,4,1,0,133,118,0.067460,0.190476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017/8/27,67,107,93.954023,35,3.178161,1.948276,0.258621,0.000000,115,2,...,10,174,0,0,3,2,151,18,0.068966,0.643678
2017/8/28,78,133,139.222749,35,4.374408,1.696682,0.151659,0.004739,101,3,...,5,211,0,0,4,2,133,72,0.056872,0.450237
2017/8/29,35,90,141.672000,35,3.888000,1.816000,0.168000,0.000000,93,0,...,3,125,0,0,2,2,119,2,0.048000,0.600000


In [7]:
test=scaler.transform(test)

In [8]:
y_test_label=pd.read_csv('test_nolabel.csv', engine='python')
# test_nolabel = pd.read_csv('test_nolabel.csv', engine='python')
# y_label_tr=pd.get_dummies(y_label, columns=["label"])
y_test_label

,arrival_date
0,2017-04-01
1,2017-04-02
2,2017-04-03
3,2017-04-04
4,2017-04-05
...,...
148,2017-08-27
149,2017-08-28
150,2017-08-29
151,2017-08-30


In [9]:
train_lstm=np.array(train)
# train_lstm=train.drop(["arrival_date"],axis=1)
train_lstm=np.array(train_lstm)
X_train = np.reshape(train_lstm, (train_lstm.shape[0], 1, train_lstm.shape[1]))
X_train, X_test, y_train, y_test = train_test_split(X_train, y_label_tr, test_size=0.2, random_state=42)
model = Sequential()
#     model.add(LSTM(units = 256, input_dim=42))
# model.add(LSTM(units = 256, input_dim=42))
# model.add(LSTM(units = 128, input_dim=42,return_sequences=True))
model.add(LSTM(units = 64, input_dim=41))
# model.add(LSTM(128))#new
# model.add(LSTM(64))#new
# model.add(Dropout(0.2))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(32, activation='relu'))
#     model.add(Dropout(0.25))
# model.add(Dense(16, activation='relu'))
model.add(Dense(10, activation='sigmoid'))
# model.add(Dense(10, activation='relu'))
callback = EarlyStopping(monitor="val_loss", patience=50, verbose=1, mode="auto")
model.compile(loss='binary_crossentropy',optimizer='adam')
model.fit(X_train, y_train, batch_size=150, epochs=1000, verbose=0,callbacks=[callback])
#     scores = model.evaluate(X_test, y_test, verbose=0)
# #     print("%s: %.2f%%" % (model.metrics_names[1], scores[1]))
#     cvscores.append(scores[1] * 100)

y_pred_train = model.predict(X_test, batch_size=64, verbose=0)
y_pred_bool_test = np.argmax(y_pred_train, axis=1)
y_test_bool = np.argmax(np.array(y_test), axis=1)
print(classification_report(y_test_bool, y_pred_bool_test))
print(confusion_matrix(y_test_bool, y_pred_bool_test))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

              precision    recall  f1-score   support

           0       0.94      0.80      0.86        20
           1       0.76      0.94      0.84        31
           2       0.68      0.62      0.65        34
           3       0.64      0.59      0.62        27
           4       0.00      0.00      0.00        10
           5       0.14      0.33      0.20         3
           6       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         1

    accuracy                           0.65       128
   macro avg       0.35      0.36      0.35       128
weighted avg       0.65      0.65      0.64       128

[[16  3  1  0  0  0  0  0  0]
 [ 1 29  1  0  0  0  0  0  0]
 [ 0  6 21  5  2  0  0  0  0]
 [ 0  0  7 16  3  0  1  0  0]

In [10]:
error = []
for i in range(len(y_test_bool)):
    error.append(y_test_bool[i] - y_pred_bool_test[i])

# print("Errors: ", error)
# print(error)

absError = []
for val in error:
    absError.append(abs(val))#誤差絕對值

# print("Absolute Value of Error: ", absError)
from math import sqrt
print("MAE = ", sum(absError) / len(absError))#平均絕對誤差MAE
 

MAE =  0.453125


In [11]:
train_lstm=np.array(train)
# train_lstm=train.drop(["arrival_date"],axis=1)
# train_lstm=np.array(train_lstm)
X_train = np.reshape(train_lstm, (train_lstm.shape[0], 1, train_lstm.shape[1]))
# X_train, X_test, y_train, y_test = train_test_split(X_train, y_label_tr, test_size=0.2, random_state=42)
model = Sequential()
#     model.add(LSTM(units = 256, input_dim=42))
# model.add(LSTM(units = 256, input_dim=42))
model.add(LSTM(units = 64, input_dim=41))
# model.add(LSTM(128))#new
# model.add(LSTM(128))#new
# model.add(LSTM(64))#new
# model.add(Dropout(0.2))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(64, activation='relu'))
#     model.add(Dropout(0.25))
#     model.add(Dense(16))
model.add(Dense(10, activation='sigmoid'))
callback = EarlyStopping(monitor="val_loss", patience=20, verbose=1, mode="auto")
model.compile(loss='binary_crossentropy',optimizer='adam')
model.fit(X_train, y_label_tr, batch_size=200, epochs=1000, verbose=0,callbacks=[callback])
#     scores = model.evaluate(X_test, y_test, verbose=0)
# #     print("%s: %.2f%%" % (model.metrics_names[1], scores[1]))
#     cvscores.append(scores[1] * 100)

y_pred_train = model.predict(X_train, batch_size=64, verbose=0)
y_pred_bool_train = np.argmax(y_pred_train, axis=1)
y_train_bool = np.argmax(np.array(y_label_tr), axis=1)
print(classification_report(y_train_bool, y_pred_bool_train))
print(confusion_matrix(y_train_bool, y_pred_bool_train))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        94
           1       1.00      1.00      1.00       152
           2       1.00      1.00      1.00       186
           3       1.00      1.00      1.00       124
           4       1.00      1.00      1.00        46
           5       1.00      1.00      1.00        21
           6       1.00      1.00      1.00        13
           7       1.00      1.00      1.00         1
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         1

    accuracy                           1.00       640
   macro avg       1.00      1.00      1.00       640
weighted avg       1.00      1.00      1.00       640

[[ 94   0   0   0   0   0   0   0   0   0]
 [  0 152   0   0   0   0   0   0   0   0]
 [  0   0 186   0   0   0   0   0   0   0]
 [  0   0   0 124   0   0   0   0   0   0]
 [  0   0   0   0  46   0   0   0   0   0]
 [  0   0   0   0   0  21 

In [12]:
test_lstm=np.array(test)
X_test = np.reshape(test_lstm, (test_lstm.shape[0], 1, test_lstm.shape[1]))
y_pred_test = model.predict(X_test, batch_size=64, verbose=0)
y_pred_bool_test = np.argmax(y_pred_test, axis=1)
y_pred_bool_test=pd.DataFrame(y_pred_bool_test,columns=["label"])
y_test_num=pd.concat([y_test_label,y_pred_bool_test],axis=1)

In [25]:
import csv

# 開啟輸出的 CSV 檔案
with open('model_predict/lstm_num_std_noqu.csv', 'w', newline='') as csvfile:
    # 建立 CSV 檔寫入器
    writer = csv.writer(csvfile)

    # 寫入標題資料
    writer.writerow(["arrival_date","label"])
    # 寫入距離資料
    for i in range(len(y_test_num)):
        writer.writerow([y_test_num["arrival_date"].iloc[i],y_test_num["label"].iloc[i]])
        

In [13]:
y_test_num.groupby("label").count()

,arrival_date
label,
2,1
3,111
4,19
5,1
6,20
8,1


In [27]:
y_label_full=pd.read_csv('train_label.csv', engine='python')

In [28]:
y_label_full.groupby("label").count()

,arrival_date
label,
0.0,94
1.0,152
2.0,186
3.0,124
4.0,46
5.0,21
6.0,13
7.0,1
8.0,2
